In [1]:
cd /tf/astromer

/tf/astromer


In [13]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sys,os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
serie_original = tf.random.normal([101, 3])
serie_random = tf.random.normal([101, 3]) # random

max_obs   = 75

In [8]:
def split_serie(serie):
    len_serie = tf.shape(serie)[0]
    padding = tf.zeros([len_serie, 1])
    if len_serie % 2 != 0:
        serie = tf.concat([serie, 
                           tf.zeros([1, tf.shape(serie)[-1]])
                          ], 0)

        padding = tf.concat([tf.zeros([len_serie, 1]),
                             tf.ones([1, 1])
                            ], 0) 

    serie_1, serie_2 = tf.split(serie, 2)
    mask_1, mask_2   = tf.split(padding, 2)
    
    return serie_1, serie_2, mask_1, mask_2

In [186]:
def cut_serie(serie, max_obs, random_pivot=False):
    size = tf.shape(serie)
    if tf.less(max_obs, size[0]):
        pivot = 0
        if random_pivot:
            pivot = tf.random.uniform(shape=[1],
                                      minval=0,
                                      maxval=int(size[0]-max_obs),
                                      dtype=tf.dtypes.int32)[0]    
            
        serie = tf.slice(serie, [pivot,0], [max_obs,-1])
        mask = tf.zeros([tf.shape(serie)[0]])
    else:
        serie = tf.concat([
            serie,
            tf.zeros([max_obs-size[0], size[-1]], dtype=tf.float32)
        ], 0)
        mask = tf.concat([
            tf.zeros([size[0], 1]),
            tf.ones([max_obs-size[0], 1])
        ], 0)
        
    return serie, mask

In [172]:
from core.data import get_masked

In [173]:
serie_1, serie_2, mask_1, mask_2 = split_serie(serie_original)

In [176]:
serie_1, mask_1 = cut_serie(serie_1, max_obs//2)
serie_2, mask_2 = cut_serie(serie_2, max_obs//2)

In [182]:
masked_values_1 = get_masked(serie_1, 0.2)
masked_values_2 = get_masked(serie_2, 0.2)

In [36]:
mask_1 = tf.maximum(mask_1, masked_values_1)
mask_2 = tf.maximum(mask_2, masked_values_2)

NameError: name 'mask_1' is not defined

In [74]:
from core.data import pretraining_records

In [75]:
dataset = pretraining_records('./data/records/macho/train', 10)

In [77]:
for batch in dataset:
    print(batch['input'])
    break

tf.Tensor(
[[[-9.9000000e+01]
  [-6.6458923e-01]
  [-7.2251447e-02]
  ...
  [ 8.3680175e-02]
  [-4.2882655e-02]
  [-9.8000000e+01]]

 [[-9.9000000e+01]
  [-4.5508128e-02]
  [ 1.1254667e+00]
  ...
  [-7.9062659e-01]
  [-4.0245226e-01]
  [-9.8000000e+01]]

 [[-9.9000000e+01]
  [ 2.6681504e+00]
  [ 1.0835730e+00]
  ...
  [-1.2305614e+00]
  [-1.0752320e-01]
  [-9.8000000e+01]]

 ...

 [[-9.9000000e+01]
  [ 8.6693835e-01]
  [ 8.3896631e-01]
  ...
  [ 4.1140761e+00]
  [ 5.9577835e-01]
  [-9.8000000e+01]]

 [[-9.9000000e+01]
  [ 4.4810906e-01]
  [ 9.8658484e-01]
  ...
  [ 9.3207592e-01]
  [ 2.2490960e-01]
  [-9.8000000e+01]]

 [[-9.9000000e+01]
  [ 2.4932218e+00]
  [ 1.1354285e+00]
  ...
  [-6.5952331e-01]
  [-6.2107795e-01]
  [-9.8000000e+01]]], shape=(10, 103, 1), dtype=float32)
